In [ ]:
from local_search import local_search_greedy
from ILS2 import *
from random_cycles import gen_random_cycles
import numpy as np
from utils import *
import matplotlib.pyplot as plt
import pickle
from calc import *
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

In [ ]:
FILENAMES = ["kroB200.tsp"]
ILS2a_TIME = 250
CYCLES_SET_SIZE = 50

results = []

for filename in FILENAMES:
    print(f"Obliczenia dla pliku {filename}")
    data = read_data_file("data/" + filename)
    dist_matrix = calculate_distance_matrix(data)

    pickle_file = 'cycles_' + filename + '.pkl'

    try:
        with open(pickle_file, 'rb') as f:
            cycles = pickle.load(f)
    except:
        print("Brak pliku z cyklami")

    c = {}

    for i in tqdm(range(len(cycles))):
        cycle1a, cycle2a = cycles[i]
        cycle1b, cycle2b = cycles[(i+1) % len(cycles)]

        common = get_common_edges(cycle1a, cycle2a, cycle1b, cycle2b)
        for edge in common:
            if edge not in c:
                c[edge] = common[edge]
            else:
                c[edge] += common[edge]
        
    print("Wspólne krawędzie policzone")
    # sort by common edges
    c = dict(sorted(c.items(), key=lambda item: item[1], reverse=True))
    c_df = pd.DataFrame(c.items(), columns=['edge', 'common'])
    c_df["length"] = c_df["edge"].apply(lambda x: dist_matrix[x[0]][x[1]])
    #len_025 = c_df["length"].quantile(0.25)
    c_df = c_df.head(int(len(c_df) * 0.08))
    #c_df = c_df[c_df["length"] <= len_025]
    c_df["a_x"] = c_df["edge"].apply(lambda x: data[x[0]][0])
    c_df["a_y"] = c_df["edge"].apply(lambda x: data[x[0]][1])
    c_df["b_x"] = c_df["edge"].apply(lambda x: data[x[1]][0])
    c_df["b_y"] = c_df["edge"].apply(lambda x: data[x[1]][1])

    # posortuj krawędzie po pierwszej współrzędnej pierwszego wierzchołka
    c_df = c_df.sort_values(by=['a_x'])

    # dodaj pierwszą połowę krawędzi do cyklu 1 i drugą do cyklu 2
    cycle1 = []
    cycle2 = []

    len_df = len(c_df)
    for i in range(len(c_df)):
        a,b = [c_df.iloc[i]["edge"][0], c_df.iloc[i]["edge"][1]]
        if i < len_df / 2:
            if a not in cycle1 and b not in cycle1:
                cycle1.append(a)
                cycle1.append(b)
        else:
            if a not in cycle2 and b not in cycle2 and a not in cycle1 and b not in cycle1:
                cycle2.append(a)
                cycle2.append(b)


    free_nodes = list(set(get_nodes(data)) - set(cycle1) - set(cycle2))

    while len(cycle1) < 100:
        best_update1 = float('inf')
        best_node1 = None
        best_position1 = -1

        for node in free_nodes:
            for i in range(len(cycle1)):
                distance_update = dist_matrix[cycle1[i-1]][node] + dist_matrix[node][cycle1[i]] - dist_matrix[cycle1[i-1]][cycle1[i]]
                if distance_update < best_update1:
                    best_update1 = distance_update
                    best_node1 = node
                    best_position1 = i

        if best_node1 is not None:
            cycle1.insert(best_position1, best_node1)
            free_nodes.remove(best_node1)


    while len(free_nodes) > 0:
        best_update2 = float('inf')
        best_node2 = None
        best_position2 = -1

        for node in free_nodes:
            for i in range(len(cycle2)):
                distance_update = dist_matrix[cycle2[i-1]][node] + dist_matrix[node][cycle2[i]] - dist_matrix[cycle2[i-1]][cycle2[i]]
                if distance_update < best_update2:
                    best_update2 = distance_update
                    best_node2 = node
                    best_position2 = i

        if best_node2 is not None:
            cycle2.insert(best_position2, best_node2)
            free_nodes.remove(best_node2)

    starting1 = deepcopy(cycle1)
    starting2 = deepcopy(cycle2)

    
    for _ in tqdm(range(1)):
        s = time.time()
        cycle1 = deepcopy(starting1)
        cycle2 = deepcopy(starting2)

        time_elapsed = 30
        no_improvement = 0
        size = 0.1
        best_length = calculate_cycles_length(cycle1, cycle2, dist_matrix)

        while time_elapsed > 0:
            start_time = time.time()
            c1, c2 = severe_perturbation(starting1, starting2, dist_matrix, size=size)
            length = calculate_cycles_length(c1, c2, dist_matrix)
            if length < best_length:
                best_length = length
                cycle1 = c1
                cycle2 = c2
            else:
                no_improvement += 1
     
                if no_improvement >= 6:
                    cycle2, cycle2 = local_search_steepest(cycle1, cycle2, dist_matrix, data)
             
                    no_improvement = 0
            
            stop_time = time.time()
            time_elapsed = time_elapsed - (stop_time - start_time)

        e = time.time()
        results.append((filename, cycle1, cycle2, calculate_cycles_length(cycle1, cycle2, dist_matrix), e-s))

results_df = pd.DataFrame(results, columns=["filename", "cycle1", "cycle2", "length", "time"])
results_df

In [ ]:
# Grupowanie danych i obliczanie statystyk
time_stats = results_df.groupby(["filename"])["time"].agg(['min', 'mean', 'max']).reset_index()
length_stats = results_df.groupby(["filename"])["length"].agg(['min', 'mean', 'max']).reset_index()

# Łączenie statystyk w jeden DataFrame
stats_df = pd.merge(time_stats, length_stats, on=["filename"], suffixes=("_time", "_length"))
stats_df

In [ ]:
# plot best cycles
results_df = results_df.sort_values(by=["filename", "length"])
best_cycles = results_df["cycle1"].iloc[0], results_df["cycle2"].iloc[0]

plot_cycles(*best_cycles, data, "Best cycles")